# Preprocessing of features for clustering

In [1]:
# import packages
import pandas as pd
import numpy as np
import psycopg2
from psycopg2 import Error
import matplotlib.pyplot as plt

Extract data from postgress

In [7]:
USER = "***"
PSWD = "***********"
HOST = "pg.analytics.northwestern.edu"
PORT = "5432"
DB_NAME = "everything2023"

In [8]:
# Connect to postgress database
try:
    # Connect to an existing database
    connection = psycopg2.connect(user = USER,
                                  password = PSWD,
                                  host = HOST,
                                  port = PORT,
                                  database = DB_NAME)

    # Create a cursor to perform database operations
    cursor = connection.cursor()
    
    # Print PostgreSQL details
    cursor.execute("SELECT version();")
    
    # Fetch result
    record = cursor.fetchone()
    
    # Print succesfull connection
    print("You are connected to - ", record, "\n")
    
except (Exception, Error) as error:
    # Print error conecting to ostgres
    print("Error while connecting to PostgreSQL", error)

You are connected to -  ('PostgreSQL 10.12 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-39), 64-bit',) 



In [9]:
# Read skuinfo table
skuinfo = pd.read_sql("select * from group_9.skuinfo", connection)
skuinfo

C:\Users\xinra\AppData\Local\Temp\ipykernel_31996\2413202832.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  skuinfo = pd.read_sql("select * from group_9.skuinfo", connection)


,SKU,DEPT,CLASSID,UPC,STYLE,COLOR,SIZE,PACKSIZE,VENDOR,BRAND
0,3,6505,113,000400000003000,00 F55KT2,WHISPERWHITE,P8EA,1,5119207,TURNBURY
1,4,8101,002,000400000004000,22 615CZ4,SPEARMI,S,1,3311144,C A SPOR
2,5,7307,003,000400000005000,7LBS 245-01,34 SILVER,KING,1,5510554,BEAU IDE
3,8,3404,00B,000400000008000,622 F05H84,MORNING MI,2T,1,2912827,HARTSTRI
4,15,2301,004,000400000015000,126 MDU461,255CAMEL,12,1,0023272,JONES/LA
...,...,...,...,...,...,...,...,...,...,...
1564173,9999973,3103,009,000400009973999,702 S3JAYV,STONE,4,1,6813115,POLO JEA
1564174,9999974,9801,726,000400009974999,G50171,NAVY MULTI,10,1,9212766,GABAR IN
1564175,9999991,2301,004,000400009991999,026 MDU201,618RED ROSE,8,1,0023272,JONES/LA
1564176,9999992,1202,402,000400009992999,14 F52UN1,PALE JADE,L,1,1446212,CABERNET


In [10]:
# Read transact table
#trnsact = pd.read_sql("select * from group_9.trnsact", connection)
#trnsact

1. Style (51 MERU08, 9 126NAO) - not used
2. Color (BLACK, KHAKI) - Daniel
3. Size (L) - Xinran
4. Brand (TOMMY HI, MARK ECK) - not used
5. Department (CLINIQUE, LESLIE) - Liang
6. Whether discount (sell price - original price <0?) - Liang
7. Most bought in which season/month (from transaction data) - Mahi
8. City/state of the store (one-to-many relationship) - Mahi
9. Return rate - Xinran

## Size

In [11]:
skuinfo["SIZE"].value_counts().head(10)

SIZE
L      125606
M      125167
S      107737
ALL    103149
XL      90229
6       31179
12      26881
14      26463
XXL     26234
10      25382
Name: count, dtype: int64

In [12]:
def convert_numerical_to_letter_size(size):
    # convert numerical sizes to int type
    try:
        size = int(size)
    # keep the letter sizes
    except (ValueError, TypeError):
        return size
    
    # covert numerical sizes to letter sizes
    if size <= 4:
        return "XS"
    elif size <= 6:
        return "S"
    elif size <= 8:
        return "M"
    elif size <= 10:
        return "L"
    elif size <= 14:
        return "XL"
    elif size <= 18:
        return "XXL"
    # return the orginal value if not converted    
    return size

def convert_xxxM_to_letter_size(size_code):
    # check if the size code is str
    if type(size_code) == str:
        # Check if the size code has at least 4 characters and ends with 'M'
        if len(size_code) >= 4 and size_code[-1] == 'M' and size_code[:-1].isdigit():

            # Extract the numeric part from the size code
            numeric_part = int(size_code[:-1])
            # Define mapping based on your specific requirements
            if numeric_part >= 100:
                return "XXL"
            elif numeric_part >= 90:
                return "XL"
            elif numeric_part >= 80:
                return "L"
            elif numeric_part >= 70:
                return "M"
            elif numeric_part >= 60:
                return "S"
            else:
                return "XS"
    
    # return the original value if not converted
    return size_code

def convert_to_other(size):
    valid_sizes = {'XS', 'S', 'M', 'L', 'XL', 'XXL', 'ALL', 'NO SIZE'}
    
    if type(size) == str:
        if size.upper() in valid_sizes:
            return size
        else:
            return 'OTHER'
    else:
        return 'OTHER'

cleaned_size = []
for size in skuinfo["SIZE"]:
    # convert 'XL   ND' to 'XL'
    if type(size) == str:
        size = size.replace('ND', '').strip()
    # convert numerical size
    size = convert_numerical_to_letter_size(size)
    # convert size in the format of xxxM 
    size = convert_xxxM_to_letter_size(size)
    # convert to other
    size = convert_to_other(size)
    cleaned_size.append(size)

pd.DataFrame(cleaned_size).value_counts()

OTHER      483712
M          200756
L          198437
XL         195121
S          191937
ALL        103175
XXL        103128
XS          84480
NO SIZE      3432
Name: count, dtype: int64

In [13]:
SKU_size = pd.DataFrame({'SKU':list(skuinfo["SKU"]),'SIZE':cleaned_size})
SKU_size.to_csv("feature_size.csv")

Convert all into XS/S/M/L/XL/XXL/ALL/NO SIZE

## Return rate

Numerical variable
use transact data, calculate the rate of return.

In [2]:
trnsact = pd.read_csv('C:/Users/xinra/Desktop/MLDS/400-everythingstartswithdata/project/data/trnsact.csv', sep=',', header= None,
                    #    dtype={'STORE':'int', 'SKU':'int', 'COST':'float', 'RETAIL':'float', 'Unknown':'int'},
                       names=['SKU', 'STORE', 'REGISTER', 'TRANNUM', 'INTERID', 'SALEDATE', 'STYPE', 'QUANTITY', 'ORGPRICE', 'SPRICE', 'AMT', 'SEQ', 'MIC', 'Unknown'])
trnsact

,SKU,STORE,REGISTER,TRANNUM,INTERID,SALEDATE,STYPE,QUANTITY,ORGPRICE,SPRICE,AMT,SEQ,MIC,Unknown
0,3,202,290,1100,326708721,2005-01-18,P,1,0.0,30.00,30.00,3500000,818,0
1,3,202,540,2700,326708721,2005-01-29,R,1,0.0,30.00,30.00,15200000,818,0
2,3,303,500,2100,23702074,2004-08-18,P,1,0.0,12.00,12.00,4600000,48,0
3,3,709,360,500,0,2005-08-14,P,1,0.0,30.00,30.00,6500000,818,0
4,3,802,660,400,0,2005-08-09,P,1,440.0,30.00,30.00,4700000,599,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120916891,9999997,7507,750,2800,0,2005-04-23,P,1,39.0,39.00,39.00,593900116,395,0
120916892,9999997,7907,790,3200,0,2005-05-03,P,1,39.0,39.00,39.00,502600061,395,0
120916893,9999997,7907,800,700,0,2005-05-04,R,1,39.0,39.00,39.00,600900066,395,0
120916894,9999997,7907,800,700,0,2005-05-04,P,1,39.0,39.00,39.00,601000066,395,0


In [3]:
trnsact = trnsact[['SKU','STYPE']]
trnsact 

,SKU,STYPE
0,3,P
1,3,R
2,3,P
3,3,P
4,3,P
...,...,...
120916891,9999997,P
120916892,9999997,P
120916893,9999997,R
120916894,9999997,P


In [4]:
# Convert 'R' to 1 and 'P' to 0
trnsact['RETURN'] = trnsact['STYPE'].apply(lambda x: 1 if x == 'R' else 0)

# Group by SKU and calculate return rate
return_rates = trnsact.groupby('SKU')['RETURN'].mean()

C:\Users\xinra\AppData\Local\Temp\ipykernel_31996\3060452069.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trnsact['RETURN'] = trnsact['STYPE'].apply(lambda x: 1 if x == 'R' else 0)


In [6]:
feature_returnrate = pd.DataFrame(return_rates)
feature_returnrate.to_csv("feature_returnrate.csv")